## Thompson sampling and the Binary Bernoulli Bandit Problem

### Introduction

### Example setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Multiarm-Bandit-TS`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Multiarm-Bandit-TS/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Multiarm-Bandit-TS/Manifest.toml`


In [2]:
# load reqd packages -
using Distributions
using Plots
using Colors
using StatsPlots
using PrettyTables

In [3]:
include("CHEME-5660-Example-CodeLib.jl");

In [4]:
# initialize the world -
world_d_dict = Dict{Int64, Bernoulli}();
K = 3;
θ_array = [0.90, 0.40, 0.70]; # values taken from Russo et al Fig. 3.1
    
# hard code the parameters for the Bernoulli distributions, these are the 
# ground truth, but the agent doesn't know them (and they are directly visible)
for k ∈ 1:K
    world_d_dict[k] = Bernoulli(θ_array[k]);
end

In [5]:
# lets build a world function -
function world(action::Int64, dict::Dict{Int64, Bernoulli})::Int64
    
    # grab the distrubution from the dict -
    d = dict[action];
    
    # roll -
    value = rand(d);
    if (value == true)
        return 1;    
    end
    
    # default -
    return 0;
end

world (generic function with 1 method)

In [6]:
# build Thompson sample object -
model = ThompsonSamplingModel()
model.K = K;
model.α = ones(K); # initialize to random values
model.β = ones(K); # initialize to random values

# recast -
world_function(a) = world(a,world_d_dict);

# setup different sample T -
T_range = range(10,stop=10000, step=100) |> collect
time_sample_results_dict = Dict{Int64, Dict{Int64,Beta}}();
for t ∈ 1:length(T_range)
    policy_dictionary_T = sample(model; 𝒯 = T_range[t], world = world_function);
    time_sample_results_dict[t] = policy_dictionary_T;
end

In [7]:
# visualize -
time_sample_results_dict

Dict{Int64, Dict{Int64, Beta}} with 100 entries:
  5  => Dict(2=>Beta{Float64}(α=1.0, β=3.0), 3=>Beta{Float64}(α=6.0, β=5.0), 1=…
  56 => Dict(2=>Beta{Float64}(α=3.0, β=6.0), 3=>Beta{Float64}(α=46.0, β=14.0), …
  35 => Dict(2=>Beta{Float64}(α=3.0, β=5.0), 3=>Beta{Float64}(α=22.0, β=8.0), 1…
  55 => Dict(2=>Beta{Float64}(α=3.0, β=5.0), 3=>Beta{Float64}(α=30.0, β=13.0), …
  60 => Dict(2=>Beta{Float64}(α=2.0, β=5.0), 3=>Beta{Float64}(α=33.0, β=11.0), …
  30 => Dict(2=>Beta{Float64}(α=3.0, β=6.0), 3=>Beta{Float64}(α=7.0, β=7.0), 1=…
  32 => Dict(2=>Beta{Float64}(α=11.0, β=7.0), 3=>Beta{Float64}(α=13.0, β=8.0), …
  6  => Dict(2=>Beta{Float64}(α=1.0, β=3.0), 3=>Beta{Float64}(α=18.0, β=8.0), 1…
  67 => Dict(2=>Beta{Float64}(α=2.0, β=4.0), 3=>Beta{Float64}(α=7.0, β=6.0), 1=…
  45 => Dict(2=>Beta{Float64}(α=1.0, β=5.0), 3=>Beta{Float64}(α=5.0, β=4.0), 1=…
  73 => Dict(2=>Beta{Float64}(α=6.0, β=8.0), 3=>Beta{Float64}(α=6.0, β=6.0), 1=…
  64 => Dict(2=>Beta{Float64}(α=14.0, β=12.0), 3=>Beta{Float

In [10]:
# build a pretty table -
# pick a time step -
T_index = 100

# initialize -
action_probability_array = Array{Any,2}(undef, K, 3);

for i ∈ 1:K
    
    action_probability_array[i,1] = i;
    action_probability_array[i,2] = θ_array[i];
    
    # get the d -
    dict = time_sample_results_dict[T_index];
    
    # grab the parameters from dict -
    dᵢ = dict[i];
    αᵢ,βᵢ = params(dᵢ);
    action_probability_array[i,3] = αᵢ/(αᵢ + βᵢ);
end

# header -
action_table_header = (["Action aᵢ", "θᵢ", "θ̂ᵢ"]);

# display table -
pretty_table(action_probability_array; header = action_table_header)

┌───────────┬─────┬──────────┐
│ Action aᵢ │  θᵢ │       θ̂ᵢ │
├───────────┼─────┼──────────┤
│         1 │ 0.9 │ 0.900273 │
│         2 │ 0.4 │      0.4 │
│         3 │ 0.7 │ 0.578947 │
└───────────┴─────┴──────────┘


### Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.